<a href="https://colab.research.google.com/github/cristina-ramos/class_NLP/blob/main/Assignments/linear_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Retrieving the dataset, untar-ing the file and importing API's

In [ ]:
!wget 'http://www.cs.cornell.edu/people/pabo/movie-review-data/'
!tar -xf scale_data.tar.gz

--2020-11-05 03:57:44--  http://www.cs.cornell.edu/people/pabo/movie-review-data/
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 52.201.128.190
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|52.201.128.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘index.html.2’

index.html.2            [ <=>                ]   7.01K  --.-KB/s    in 0s      

2020-11-05 03:57:44 (139 MB/s) - ‘index.html.2’ saved [7176]



In [ ]:
import numpy as np
import pandas as pd
import glob
import string
from sklearn import datasets, linear_model, metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, max_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

> Creating a single dataset for classification from the IDs. I'm using Pandas I know that's not your favorite oops

In [ ]:
entries = glob.glob('/content/scaledata/*')
entries.sort()

ratings = []
subject = []
labels = []
id = []

for entry in entries:
  set_of_entries = glob.glob(entry+'/*')
  set_of_entries.sort()
  for file in set_of_entries:
    if 'id' in file:
      id_list = open(file, 'r').read().split('\n')
      id_list.pop()
    elif 'rating' in file:
      ratings_list = open(file, 'r').read().split('\n')
      ratings_list.pop()
    elif 'subj' in file:
      subject_list = open(file, 'r').read().split('\n')
      subject_list.pop()
    elif '3class' in file:
      label_list = open(file, 'r').read().split('\n')
      label_list.pop()
  id.extend(id_list)
  ratings.extend(ratings_list)
  subject.extend(subject_list)
  labels.extend(label_list)

In [ ]:
data = pd.DataFrame()
data['ratings'] = ratings
data['subject'] = subject
data['labels'] = labels
data['id'] = id
# print(data['ratings'])

> Next: splitting the datasets into train and test sets using sklearn

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=2)

> Using CountVectorizer to generate feats from subject and labels

In [ ]:
vect = CountVectorizer()
X_train = vect.fit_transform(train['subject']).toarray()
y_train = train['labels'].to_list()
X_test = vect.transform(test['subject']).toarray()
y_test = test['labels'].to_list()
ystr_train = train['ratings'].to_list()
yf_train = [float(i) for i in ystr_train]
ystr_test = test['ratings'].to_list()
yf_test = [float(i) for i in ystr_test]

> Using sklearn's SGD Regressor

In [ ]:
reg = SGDRegressor(max_iter=100)
reg.fit(X_train, yf_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=100,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [ ]:
predict = reg.predict(X_test)
errors = max_error(yf_test, predict)
print(errors)

84049408797.2691


> Now for the SGD Classifier: I had to keep lowering the max iteration because it was taking up to 30 minutes at one point just to run block 41...not sure if that's just because of the size or something I'm doing wrong on my end

In [ ]:
clf = SGDClassifier(loss="log", penalty="l1", max_iter=50)
clf.fit(X_train, y_train)
#this takes a lot longer than the l2 reg, I tried using sparsify() since it's recommended to make it easier to process L1 regularization but I didn't have much luck

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=50,
              n_iter_no_change=5, n_jobs=None, penalty='l1', power_t=0.5,
              random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
#performance of the classifier with L1 regularization:
clf_predict = clf.predict(X_test)
print("Performance of Classifier with L1 is: \n" + metrics.classification_report(y_test, clf_predict))

Performance of Classifier with L1 is: 
              precision    recall  f1-score   support

           0       0.59      0.66      0.62       230
           1       0.59      0.55      0.57       373
           2       0.76      0.75      0.75       399

    accuracy                           0.65      1002
   macro avg       0.64      0.65      0.65      1002
weighted avg       0.65      0.65      0.65      1002



In [ ]:
# clf.n_iter_
#just checking out of curiosity from the discussion in class

> Performance of the Classifier on the test dataset with L2 regularization:

In [ ]:
clf = SGDClassifier(loss="log", penalty="l2", max_iter=50)
clf.fit(X_train, y_train)
print("Performance of Classifier with L2: \n" + metrics.classification_report(y_test, clf.predict(X_test)))

Performance of Classifier with L2: 
              precision    recall  f1-score   support

           0       0.66      0.60      0.63       230
           1       0.61      0.58      0.60       373
           2       0.72      0.78      0.75       399

    accuracy                           0.67      1002
   macro avg       0.66      0.66      0.66      1002
weighted avg       0.66      0.67      0.66      1002



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


> L2 outperforms L1 which had a 0.45 accuracy score (or at least that was the original score when I first ran the code, again not too sure why the scores keep changing)

In [ ]:
# clf_l2.n_iter_

46

> Evaluation scores through sklearn's cross validation cross_val_score function

In [ ]:
parameters = [0.0001, 0.001, 0.015, 0.05]

for k in parameters:
  clf = SGDClassifier(loss="log", max_iter=100, alpha=k)
  scores = cross_val_score(clf, X_train, y_train, cv=None)
  print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


[0.64669164 0.61423221 0.67790262 0.64918851 0.6225    ]


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


[0.6454432  0.63545568 0.67041199 0.64419476 0.625     ]
[0.65418227 0.64918851 0.66042447 0.66292135 0.63625   ]
[0.65667915 0.62796504 0.68289638 0.65667915 0.63625   ]


> Using different loss function: perceptron. This performed about the same as the other functions

In [ ]:
ptn = SGDClassifier(loss="perceptron", max_iter=50)
ptn.fit(X_train, y_train)
print("Perceptron Classifier Report: \n" + metrics.classification_report(y_test, ptn.predict(X_test)))

Perceptron Classifier Report: 
              precision    recall  f1-score   support

           0       0.61      0.69      0.65       230
           1       0.62      0.58      0.60       373
           2       0.76      0.75      0.76       399

    accuracy                           0.67      1002
   macro avg       0.66      0.67      0.67      1002
weighted avg       0.67      0.67      0.67      1002



> What kind of multi-class strategy does SGDClassifier use?
  
  SGDClassifier uses a multinomial log regression

In [ ]:
Hyperplane: